# __Brough Lecture Notes: GARCH Models - Estimation via MLE__

<br>

Finance 5330: Financial Econometrics <br>
Tyler J. Brough <br>
Last Updated: March 28, 2019 <br>
<br>
<br>

These notes are based in part on the excellent monograph [Introduction to Python for Econometrics, Statistics, and Data Analysis](https://www.kevinsheppard.com/images/b/b3/Python_introduction-2016.pdf) by the econometrician Kevin Sheppard of Oxford Univeristy. Many thanks to Dr. Sheppard for making his lecture material publically available. 

<br>

## Estimating GARCH Models

The standard procedure to fit GARCH models to historical returns time series data is to implement a numerical _maximum likelihood estimation_ (MLE) method. 

<br>

__NB:__ though [Bayesian](https://www.springer.com/us/book/9783540786566) methods have been shown to be superior!

<br>

The typical setup is as follows: 

* Continuously compounded returns are assumed to have a conditionally normal distribution $N(0, \sigma_{t})$

* We can estimate the GARCH parameter weights via a numerical optimization routine such as Nelder-Mead or Newton-Raphson.

* That is, the numerical routine searches for the parameter values that maximizes the value of the likelihood function.

<br>

Under the normality assumption the probility density of $\epsilon_{t}$, conditional on $\sigma_{t}$, is 

<br>

$$
f(\epsilon | \sigma_{t}) = \frac{1}{\sqrt{2\pi \sigma_{t}}} e^{-0.5  \frac{\epsilon_{t}^{2}}{\sigma_{t}}}
$$

<br>

Since the $\epsilon_{t}$ are conditionally independent, the probability of observing the actual returns that are observed is  the product of the probabilities, this is given by the likelihood function:

<br>

$$
\prod\limits_{t=1}^{T} f(\epsilon_{t} | \sigma_{t}) = \prod\limits_{t=1}^{T} \left( \frac{1}{\sqrt{2\pi \sigma_{t}}} e^{-0.5  \frac{\epsilon_{t}^{2}}{\sigma_{t}}} \right)
$$

<br>

For the GARCH(1,1) model, $\sigma_{t}$ is a function of $\omega$, $\alpha$, and $\beta$. The MLE will select values for these parameters $\hat{\omega}$, $\hat{\alpha}$, and $\hat{\beta}$ - that maximize the value of the probability of observing the returns we actually historically did observe. 

<br>

Typically, it is easiest to maximize the value of the log-likelihood function as follows: 

<br>

$$
\sum\limits_{t=1}^{T} \left[ -0.5 \ln{(\sigma_{t})} - 0.5 \frac{\epsilon_{t}^{2}}{\sigma_{t}}\right] = -0.5 \sum\limits_{t=1}^{T} \left[\ln{(\sigma_{t})} + \frac{\epsilon_{t}^{2}}{\sigma_{t}}\right]
$$

We can omit the term $-0.5 \ln{(2 \pi)}$ since it does not affect the solution. Though in some software it is left in. 

<br>

We can implement this MLE estimation in Python by utilizing the [optimize](https://docs.scipy.org/doc/scipy/reference/optimize.html) module in [Scipy](https://docs.scipy.org/doc/scipy/reference/index.html), which contains a host of numerical optimization routines. 

<br>

First, we need to define a function to implement the log-likelihood function.

<br>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [20, 10]
from numpy import size, log, exp, pi, sum, diff, array, zeros, diag, mat, asarray, sqrt, copy
from numpy.linalg import inv
import scipy.optimize as opt

In [3]:
def garch_likelihood(params, data, sigma2, out=None):
    mu = params[0]
    omega = params[1]
    alpha = params[2]
    beta = params[3]
    
    T = size(data, 0)
    eps = data - mu
    
    for t in range(1, T):
        sigma2[t] = omega + alpha * eps[t-1]**2 + beta * sigma2[t-1]
        
    lls = 0.5 * (log(2 * pi) + log(sigma2) + eps**2/sigma2)
    ll = sum(lls)
    
    if out is None:
        results = ll
    else:
        results = (ll, lls, copy(sigma2))
        
    return results

<br>

We should begin with data simulated from the model as a first-run test case. So, let's import the simulation code.

<br>

In [4]:
## Set GARCH parameters (these might come from an estimated model)
mu = log(1.15) / 252
w = 10.0**-6
a = 0.085
b = 0.905

In [5]:
np.sqrt(w / (1.0 - a - b)) * np.sqrt(252)

0.15874507866387536

In [6]:
def simulate_garch(parameters, numObs):
    
    ## extract the parameter values
    mu = parameters[0]
    w = parameters[1]
    a = parameters[2]
    b = parameters[3]
    
    ## initialize arrays for storage
    z = np.random.normal(size=(numObs + 1))
    q = zeros((numObs + 1))
    r = zeros((numObs + 1))
    
    ## fix initial values 
    q[0] = w / (1.0 - a - b)
    r[0] = mu + z[0] * sqrt(q[0])
    e = (r[0] - mu) 
    
    ## run the main simulation loop
    for t in range(1, numObs + 1):
        q[t] = w + a * (e * e) + b * q[t-1]
        r[t] = mu + z[t] * sqrt(q[t])
        e = (r[t] - mu) 
        
    ## return a tuple with both returns and conditional variances
    return (r, q)

In [9]:
## number of trading days per year
numObs = 2500

## daily continuously compounded rate of return correpsonding to 15% annual
mu = log(1.15) / 252
#mu = 0.0

## drift and GARCH(1,1) parameters in an array
params = array([mu, w, a, b])

## run the simulation
r, s = simulate_garch(params, numObs)

<br>

Okay, now we will set up the estimation of the model on this simulated data. The goal is to see if the numerical search routine embedded in the maximum likelihood estimation
can recapture the preset parameter weights given above. 

<br>

It is important to test our software with a known scenario like this before we take the model to real-world data first as a sanity check. Otherwise we could end up chasing our tails for hours and hours without effect. 

<br>

The numerical algorithm requires good starting values as initial conditions for the search to begin. Here we will follow Sheppard. (Notice that this feels "prior" like, as in the Bayesian sense) A more data-based way to do this is to use some kind of grid search to find values that have "small" log-likelihood values. 

<br>

In [10]:
r.mean()

0.0004360455203405867

In [11]:
#begVals = array([r.mean(), r.var() * .01, .09, .90])
begVals = array([mu, w, a, b])

In [30]:
finfo = np.finfo(np.float64)
bounds = [(-10 * r.mean(), 10 * r.mean()), (finfo.eps, 2 * r.var()), (0.0, 1.0), (0.0, 1.0)]
#results = opt.minimize(fun=garch_likelihood, x0=begVals, args=(r,s), method='L-BFGS-B', bounds=bounds)
#results = opt.minimize(fun=garch_likelihood, x0=begVals, args=(r,s), method='Nelder-Mead')
#results = opt.minimize(fun=garch_likelihood, x0=begVals, args=(r,s), method='BFGS')
results = opt.minimize(fun=garch_likelihood, x0=begVals, args=(r,s), method='SLSQP', bounds=bounds)

In [31]:
results

     fun: -8590.889783346158
     jac: array([ 3.69059167e+03, -2.96363403e+06,  1.74062988e+02, -4.21751709e+01])
 message: 'Positive directional derivative for linesearch'
    nfev: 35
     nit: 8
    njev: 4
  status: 8
 success: False
       x: array([5.46225037e-04, 1.04135801e-06, 8.48182848e-02, 9.02941056e-01])

In [29]:
vals = results['x']
print(f"The estimated mean return is: {vals[0] : 0.8f} vs {mu : 0.8f}")
print(f"The estimated omega value is: {vals[1] : 0.8f} vs {w : 0.8f}")
print(f"The estimated alpha value is: {vals[2] : 0.8f} vs {a : 0.08f}")
print(f"The estimated beta values is: {vals[3] : 0.8f} vs {b : 0.08f}")

The estimated mean return is:  0.00054406 vs  0.00055461
The estimated omega value is:  0.00000146 vs  0.00000100
The estimated alpha value is:  0.08245276 vs  0.08500000
The estimated beta values is:  0.89794951 vs  0.90500000


In [23]:
r.std(ddof=1) * np.sqrt(252)

0.13019864666332087

In [24]:
np.sqrt(vals[0] / (1.0 - vals[1] - vals[2])) * np.sqrt(252)

0.3563749517218149

<br>

___So much depends upon those initial guesses!___

<br>

## DEM2GBP GARCH(1,1) Model Estimation via MLE

In [72]:
df = pd.read_csv("./data/dem2gbp.csv")

In [73]:
df.head()

,DEM2GBP
0,0.125333
1,0.028874
2,0.063462
3,0.226719
4,-0.214267


In [74]:
df.tail()

,DEM2GBP
1969,-0.408540
1970,-0.030468
1971,-0.117546
1972,-0.231271
1973,0.528047


In [90]:
r = df.DEM2GBP.values[:749] 
sigma = np.ones_like(r) * r.var()
args = (r, sigma)

begVals = array([0.0, 0.045, .23, .64])
finfo = np.finfo(np.float64)
bounds = [(-10, 10), (finfo.eps, 2 * r.var()), (0.0, 1.0), (0.0, 1.0)]
results = opt.minimize(fun=garch_likelihood, x0=begVals, args=args, method='SLSQP', bounds=bounds) #, constraints=garch_constraints)

In [91]:
results

     fun: 581.1073109492175
     jac: array([-0.01179504, -0.13036346, -0.02046204, -0.03572845])
 message: 'Optimization terminated successfully.'
    nfev: 57
     nit: 8
    njev: 8
  status: 0
 success: True
       x: array([-0.0262441 ,  0.04480614,  0.21335949,  0.6518363 ])

In [92]:
results = opt.minimize(fun=garch_likelihood, x0=begVals, args=args, method='L-BFGS-B', bounds=bounds)

In [93]:
results

      fun: 581.1073108069043
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.01693934, -0.01446097,  0.00229647, -0.000216  ])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 100
      nit: 15
   status: 0
  success: True
        x: array([-0.02623539,  0.04480419,  0.21335574,  0.65185522])

In [94]:
results = opt.minimize(fun=garch_likelihood, x0=begVals, args=args, method='BFGS')

In [95]:
results

      fun: 581.1073107498528
 hess_inv: array([[ 1.05336001e-03,  2.88254025e-04, -1.37384236e-05,
        -1.05420765e-03],
       [ 2.88254025e-04,  1.91953218e-04, -5.41839290e-06,
        -6.90485243e-04],
       [-1.37384236e-05, -5.41839290e-06,  2.06073382e-05,
         7.47994636e-06],
       [-1.05420765e-03, -6.90485243e-04,  7.47994636e-06,
         2.75167996e-03]])
      jac: array([1.52587891e-05, 1.52587891e-05, 0.00000000e+00, 0.00000000e+00])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 121
      nit: 13
     njev: 20
   status: 2
  success: False
        x: array([-0.02624089,  0.04480515,  0.21335265,  0.65185394])

In [96]:
results = opt.minimize(fun=garch_likelihood, x0=begVals, args=args, method='Nelder-Mead')

In [97]:
results

 final_simplex: (array([[-0.02623616,  0.04480605,  0.21336221,  0.65186079],
       [-0.02622822,  0.04479104,  0.21334922,  0.65191292],
       [-0.02622547,  0.04481243,  0.21341521,  0.65177775],
       [-0.02626151,  0.04480939,  0.21330421,  0.65187475],
       [-0.02625828,  0.0447872 ,  0.2133712 ,  0.65190942]]), array([581.10731131, 581.10731187, 581.10731192, 581.10731239,
       581.10731296]))
           fun: 581.1073113149031
       message: 'Optimization terminated successfully.'
          nfev: 166
           nit: 97
        status: 0
       success: True
             x: array([-0.02623616,  0.04480605,  0.21336221,  0.65186079])